In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# cd /content/drive/MyDrive/

In [ ]:
import pandas as pd

train_df = pd.read_csv('MURA-v1.1/train_labeled_studies.csv')
valid_df = pd.read_csv('MURA-v1.1/valid_labeled_studies.csv')

train_df.head(), valid_df.head()

(  MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/  1
 0  MURA-v1.1/train/XR_SHOULDER/patient00002/study...         1
 1  MURA-v1.1/train/XR_SHOULDER/patient00003/study...         1
 2  MURA-v1.1/train/XR_SHOULDER/patient00004/study...         1
 3  MURA-v1.1/train/XR_SHOULDER/patient00005/study...         1
 4  MURA-v1.1/train/XR_SHOULDER/patient00006/study...         1,
   MURA-v1.1/valid/XR_WRIST/patient11185/study1_positive/  1
 0  MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...      1
 1  MURA-v1.1/valid/XR_WRIST/patient11186/study2_p...      1
 2  MURA-v1.1/valid/XR_WRIST/patient11186/study3_p...      1
 3  MURA-v1.1/valid/XR_WRIST/patient11187/study1_p...      1
 4  MURA-v1.1/valid/XR_WRIST/patient11188/study1_p...      1)

In [ ]:
def prepare_data(df):
    data = []
    for index, row in df.iterrows():
        image_path = row[0]  # Assuming the first column contains the image paths
        label = row[1]       # Assuming the second column contains the labels
        data.append((image_path, label))
    return data

train_data = prepare_data(train_df)
valid_data = prepare_data(valid_df)

len(train_data), len(valid_data), train_data[:5], valid_data[:5]

/scratch/5671805.1.csgpu/ipykernel_1034708/957062800.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = row[0]  # Assuming the first column contains the image paths
/scratch/5671805.1.csgpu/ipykernel_1034708/957062800.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[1]       # Assuming the second column contains the labels


(13456,
 1198,
 [('MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/', 1),
  ('MURA-v1.1/train/XR_SHOULDER/patient00003/study1_positive/', 1),
  ('MURA-v1.1/train/XR_SHOULDER/patient00004/study1_positive/', 1),
  ('MURA-v1.1/train/XR_SHOULDER/patient00005/study1_positive/', 1),
  ('MURA-v1.1/train/XR_SHOULDER/patient00006/study1_positive/', 1)],
 [('MURA-v1.1/valid/XR_WRIST/patient11186/study1_positive/', 1),
  ('MURA-v1.1/valid/XR_WRIST/patient11186/study2_positive/', 1),
  ('MURA-v1.1/valid/XR_WRIST/patient11186/study3_positive/', 1),
  ('MURA-v1.1/valid/XR_WRIST/patient11187/study1_positive/', 1),
  ('MURA-v1.1/valid/XR_WRIST/patient11188/study1_positive/', 1)])

In [ ]:
from collections import defaultdict

def divide_by_body_part(data):
    body_parts_data = defaultdict(list)
    for path, label in data:
        body_part = path.split('/')[2]  # Assuming the third component is the body part
        body_parts_data[body_part].append((path, label))
    return body_parts_data

train_data_by_body_part = divide_by_body_part(train_data)
valid_data_by_body_part = divide_by_body_part(valid_data)

train_body_parts = {part: len(train_data_by_body_part[part]) for part in train_data_by_body_part}
valid_body_parts = {part: len(valid_data_by_body_part[part]) for part in valid_data_by_body_part}

train_body_parts, valid_body_parts

({'XR_SHOULDER': 2820,
  'XR_HUMERUS': 592,
  'XR_FINGER': 1935,
  'XR_ELBOW': 1754,
  'XR_WRIST': 3460,
  'XR_FOREARM': 877,
  'XR_HAND': 2018},
 {'XR_WRIST': 236,
  'XR_FOREARM': 133,
  'XR_HAND': 167,
  'XR_HUMERUS': 135,
  'XR_SHOULDER': 194,
  'XR_ELBOW': 158,
  'XR_FINGER': 175})

In [ ]:
train = {}
valid = {}

def create_dfs_by_body_part(data_by_body_part, dfs_by_body_part):
    for body_part, data in data_by_body_part.items():
        df = pd.DataFrame(data, columns=['Image_Path', 'Label'])
        dfs_by_body_part[body_part] = df

create_dfs_by_body_part(train_data_by_body_part, train)
create_dfs_by_body_part(valid_data_by_body_part, valid)

train['XR_SHOULDER'].head(), valid['XR_SHOULDER'].head()

(                                          Image_Path  Label
 0  MURA-v1.1/train/XR_SHOULDER/patient00002/study...      1
 1  MURA-v1.1/train/XR_SHOULDER/patient00003/study...      1
 2  MURA-v1.1/train/XR_SHOULDER/patient00004/study...      1
 3  MURA-v1.1/train/XR_SHOULDER/patient00005/study...      1
 4  MURA-v1.1/train/XR_SHOULDER/patient00006/study...      1,
                                           Image_Path  Label
 0  MURA-v1.1/valid/XR_SHOULDER/patient11676/study...      1
 1  MURA-v1.1/valid/XR_SHOULDER/patient11703/study...      1
 2  MURA-v1.1/valid/XR_SHOULDER/patient11344/study...      1
 3  MURA-v1.1/valid/XR_SHOULDER/patient11281/study...      1
 4  MURA-v1.1/valid/XR_SHOULDER/patient11368/study...      1)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import os

torch.manual_seed(42)

class MURADataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.transform = transform
        self.data_list = []

        for body_part, data in data_dict.items():
            for _, row in data.iterrows():
                img_dir = os.path.dirname(row['Image_Path'])
                for img_file in os.listdir(img_dir):
                    if img_file.startswith('.'):
                        continue
                    img_path = os.path.join(img_dir, img_file)
                    if os.path.isfile(img_path):
                        # Include body part information in the data_list
                        self.data_list.append((img_path, row['Label'], body_part))

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        img_path, label, body_part = self.data_list[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label, body_part


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix
import torch
import numpy as np
from tqdm import tqdm
import time

def train_model(model, train_loader, criterion, optimizer, device, model_path, num_epochs=10):
    model.train()

    # Lists to keep track of progress
    epoch_loss_history = []
    epoch_acc_history = []
    start_time = time.time()

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels, _ in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total

        epoch_loss_history.append(epoch_loss)
        epoch_acc_history.append(epoch_acc.item())

        print(f'Epoch {epoch + 1}/{num_epochs} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        if epoch % 20 == 0:
            torch.save(model.state_dict(), f'{model_path}_epoch_{epoch}_model.pth')

    elapsed_time = time.time() - start_time
    return model, epoch_loss_history, epoch_acc_history, elapsed_time


def validate_model(model, valid_loader, device, model_path):
    model.eval()  # Set model to evaluate mode

    all_preds = []
    all_labels = []
    parts_list = []  # To store body part information for each prediction

    with torch.no_grad():
        for inputs, labels, parts in tqdm(valid_loader):  # Assuming body part info is available
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            parts_list.extend(parts)  # Assuming 'parts' is a list or similar iterable
    #Calculate Values for Sensitivity and Specificity graph
    fpr, tpr, thresholds = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)
    plot_roc_curve(fpr, tpr, roc_auc, model_path)
    # Calculate metrics for each body part
    parts_set = set(parts_list)  # Get unique body parts
    body_part_metrics = {}

    for part in parts_set:
        part_indices = [i for i, p in enumerate(parts_list) if p == part]  # Indices of this part
        part_preds = np.array([all_preds[i] for i in part_indices])
        part_labels = np.array([all_labels[i] for i in part_indices])

        tn, fp, fn, tp = confusion_matrix(part_labels, part_preds).ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
        accuracy = (tp + tn) / (tp + tn + fp + fn)

        body_part_metrics[part] = {
            'sensitivity': round(sensitivity, 2),
            'specificity': round(specificity, 2),
            'accuracy': round(accuracy, 2)
        }

    # Calculate overall metrics
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    overall_sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    overall_specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    overall_accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Prepare data to save
    metrics_data = [
        ['Body Part', 'Sensitivity', 'Specificity', 'Accuracy']
    ]

    for part, metrics in body_part_metrics.items():
        metrics_data.append([part, metrics['sensitivity'], metrics['specificity'], metrics['accuracy']])

    metrics_data.append(['Overall', round(overall_sensitivity, 2), round(overall_specificity, 2), round(overall_accuracy, 2)])

    metrics_data.append(['fpr', 'tpr', 'thresholds', 'roc_auc'])

    for i in range(len(fpr)):
        metrics_data.append([fpr[i], tpr[i], thresholds[i], roc_auc])


    return metrics_data


def plot_training_history(epoch_loss_history, epoch_acc_history):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epoch_loss_history, label='Loss')
    plt.title('Loss vs. Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(range(len(epoch_loss_history)))
    plt.legend()

    plt.subplot(1, 2, 2)
    print('epoch_acc_history:', epoch_acc_history)
    plt.plot(epoch_acc_history, label='Accuracy')

    plt.title('Accuracy vs. Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(range(len(epoch_acc_history)))
    plt.legend()

    plt.show()

def plot_roc_curve(fpr, tpr, roc_auc, model_path):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig(f'{model_path}_roc_curve.png')
    plt.close()  # Close the plot to avoid overlapping plots
    # plt.show()

In [ ]:
#Define Hyperparameters
# SAVE_PATH = '/content/drive/MyDrive/EC523_Project/Models/Model_Architectures'
SAVE_PATH = './Models/Model_Architectures'
learning_rate = 0.0001
momentum = 0.9
batch_size = 8
num_epochs = 100
crit = "CrossEntropy"
opti = "SGD"
transformations = "ReSize: 320X320, Rotation: 30deg, Vert_Flip: 50%, Hor_Flip: 50%, Normalization:(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define a transformation pipeline with normalization
train_transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Initialize the pretrained list of models

models_list = [models.densenet121, models.densenet161, models.densenet169, models.densenet201,
               models.resnext101_32x8d, models.resnext101_64x4d, models.resnet152, models.resnet50]

criterion = nn.CrossEntropyLoss()

train_dataset = MURADataset(train, transform=train_transform)
valid_dataset = MURADataset(valid, transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 10)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers = 10)

In [ ]:
import csv

def save_history_to_csv(model_name, train_loss_history, train_acc_history, path):
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Train Loss', 'Train Accuracy'])
        for epoch, (loss, acc) in enumerate(zip(train_loss_history, train_acc_history), 1):
            writer.writerow([epoch, loss, acc])

def save_model_training_parameters_to_csv(model_name,transformations,optimizer,momentum,batch_size,num_epochs, elapsed_time, path):
  with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Model Name', 'Transformations', 'Optimizer', 'Learning Rate', 'Momentum', 'Batch Size', 'Number of Epochs', 'Training Time'])
        writer.writerow([model_name, transformations, optimizer, learning_rate, momentum, batch_size, num_epochs, elapsed_time])

def save_validation_to_csv(metrics_data, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(metrics_data)


In [ ]:
for model_fn in models_list:
    model_name = model_fn.__name__
    model = model_fn(pretrained=True)
    model = model.to(device)

    # Create paths for csv files
    model_folder = os.path.join(SAVE_PATH, model_name)
    model_path = os.path.join(model_folder, model_name)

    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    train_path = os.path.join(model_folder, 'train_history.csv')
    train_parameters_path = os.path.join(model_folder, 'train_parameters.csv')
    model_results_path = os.path.join(model_folder, 'validation_results.csv')

    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    model, train_loss_history, train_acc_history,elapsed_time = train_model(model, train_loader, criterion, optimizer, device, model_path, num_epochs=num_epochs)
    metrics_data = validate_model(model, valid_loader, device, model_path)

    # Save training results to CSV to their respective paths
    torch.save(model.state_dict(), f'{model_path}_final_model.pth')
    save_history_to_csv(model_name, train_loss_history, train_acc_history,train_path)
    save_model_training_parameters_to_csv(model_name,transformations,opti,momentum,batch_size,num_epochs,elapsed_time, train_parameters_path)
    save_validation_to_csv(metrics_data, model_results_path)